# Das elektrische Feld in einem Kondensator

Wird durch die Poissongleichung beschrieben

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

In [ ]:
def MakeMesh():
    square = MoveTo(0,0).RectangleC(20,20).Face()
    el1 = MoveTo(0,1).RectangleC(5,0.5).Face()
    el1.edges.name="el1"
    el1.vertices.hpref=1
    el2 = MoveTo(0,-1).RectangleC(5,0.5).Face()
    el2.edges.name="el2"
    el2.vertices.hpref=1
    geo = square - el1 - el2
    mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=2))
    # mesh.RefineHP(3, 0.25)
    return geo,mesh

Wir erzeugen eine Geometrie und FiniteElemente Netz eines Plattenkondensators:

In [ ]:
geo, mesh = MakeMesh()
Draw (geo)
Draw (mesh);

Wir assemblieren und lösen das Finite Elemente Gleichungssytem. Das Potential auf den Platten (+1 Volt, -1 Volt) wird als Randbedingung vorgegeben:

In [ ]:
def SolveProblem(mesh):
    fes = H1(mesh, order=5, dirichlet="el.*")
    u,v = fes.TnT()

    a = BilinearForm(grad(u)*grad(v)*dx).Assemble()

    gfu = GridFunction(fes)
    gfu.Interpolate( mesh.BoundaryCF( {"el1":1, "el2":-1 }), mesh.Boundaries(".*"))

    inv = a.mat.Inverse(freedofs=fes.FreeDofs())
    gfu.vec.data -= inv@a.mat * gfu.vec
    return gfu

gfu = SolveProblem(mesh)
Draw (gfu, deformation=True, scale=5);

Das elektrische Feld ist der Gradient des elektrischen Potentials:

In [ ]:
Draw (Norm(grad(gfu)), mesh, order=3, deformation=True, min=0, max=3);

Um die Spitzen (Singularitäten, Unstetigkeitsstellen) des elektrischen Feldes besser auflösen zu können müssen wir das Netz dort lokal verfeinern. Dazu aktivieren wir das `RefineHP`

In [ ]:
geo, mesh = MakeMesh()
mesh.RefineHP(3, 0.25)
gfu = SolveProblem(mesh)
Draw (gfu, deformation=True, scale=5)
Draw (Norm(grad(gfu)), mesh, order=3, deformation=True, min=0, max=3);

Um solche Singularitäten zu vermeiden können wir die eckige Geometrie abrunden:

In [ ]:
def MakeMeshRounded():
    square = MoveTo(0,0).RectangleC(20,20).Face()
    el1 = MoveTo(0,1).RectangleC(5,0.5).Face()
    el1 += MoveTo(2.5,1).Circle(0.25).Face()
    el1 += MoveTo(-2.5,1).Circle(0.25).Face()
    el1.edges.name="el1"
    el2 = MoveTo(0,-1).RectangleC(5,0.5).Face()
    el2 += MoveTo(2.5,-1).Circle(0.25).Face()
    el2 += MoveTo(-2.5,-1).Circle(0.25).Face()
    el2.edges.name="el2"
    geo = square - el1 - el2
    mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=2))
    return geo,mesh

In [ ]:
geo, mesh = MakeMeshRounded()
Draw(geo)
Draw (mesh);

In [ ]:
gfu = SolveProblem(mesh)
Draw (gfu, deformation=True, scale=5)
Draw (Norm(grad(gfu)), mesh, order=3, deformation=True, min=0, max=2);

Wir sehen aber noch immer Spitzen in den Ecken des Netzes. Um diese hausgemachten Unstetigkeitsstellen zu vermeiden krümmen wir die Elemente mit der `Curve` Funktion.

In [ ]:
geo, mesh = MakeMeshRounded()
mesh.Curve(5)

gfu = SolveProblem(mesh)
Draw (gfu, deformation=True, scale=5)
Draw (Norm(grad(gfu)), mesh, order=3, deformation=True, min=0, max=3);